In [3]:
from cloudflare import Cloudflare
from dotenv import load_dotenv
import os 
load_dotenv()

import logging
logging.basicConfig(force=True, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def update_dns_records():
    cloudflare_api_global_key = os.getenv('CLOUDFLARE_GLOBAL_KEY') # this must be the global key, not a token!
    cloudflare_email = os.getenv('CLOUDFLARE_EMAIL')
    # assume that the zone and record name are the same.
    zone_and_record_names = [
        'jasonmcaffee.com',
        'yogajhana.com'
    ]
    new_ip = '73.63.43.11' #TODO get this from the router due to vpn, and not wanting to rely on a 3rd party.
    cloudflare_client = Cloudflare(api_key=cloudflare_api_global_key, api_email=cloudflare_email)
    
    for zone_and_record_name in zone_and_record_names:
        update_dns_record(cloudflare_client, zone_and_record_name, zone_and_record_name, new_ip)
    
def update_dns_record(cf, zone_name, record_name, new_ip):
    try:
        logging.info(f'getting list of zones for zone_name: {zone_name}')
        zones = cf.zones.list(name=zone_name)
        if not zones:
            logging.error(f"❌ Zone {zone_name} not found. ❌")
            return
        zone_1 = zones.result[0]
        zone_id = zone_1.id
    except Exception as e:
        logging.error(f"❌ Failed to get zones: {e} ❌")
        return

    # Get the DNS record ID
    try:
        logging.info(f'getting dns records for zone and record name {record_name}')
        dns_records = cf.dns.records.list(zone_id=zone_id, name=record_name)
        if not dns_records:
            logging.error(f"❌ DNS record {record_name} not found. ❌")
            return
        record_id = dns_records.result[0].id
    except Exception as e:
        logging.error(f"❌ Failed to get DNS records: {e} ❌")
        return

    try:
        logging.info(f'updating dns record with ip: {new_ip}')
        cf.dns.records.update( zone_id=zone_id, content=new_ip, type='A', name=record_name, dns_record_id=record_id, proxied=True)
        logging.info("✅ DNS record updated successfully! ✅")
    except Exception as e:
        print(f"❌ Failed to update DNS record: {e} ❌")
        
update_dns_records()  

2024-08-05 15:24:40,987 - INFO - getting list of zones for zone_name: jasonmcaffee.com
2024-08-05 15:24:44,316 - INFO - HTTP Request: GET https://api.cloudflare.com/client/v4/zones?name=jasonmcaffee.com "HTTP/1.1 200 OK"
2024-08-05 15:24:44,317 - INFO - getting dns records for zone and record name jasonmcaffee.com
2024-08-05 15:24:44,493 - INFO - HTTP Request: GET https://api.cloudflare.com/client/v4/zones/587f1b42c95fa1e9343f558b7d55252e/dns_records?name=jasonmcaffee.com "HTTP/1.1 200 OK"
2024-08-05 15:24:44,495 - INFO - updating dns record with ip: 73.63.43.11
2024-08-05 15:24:44,647 - INFO - HTTP Request: PUT https://api.cloudflare.com/client/v4/zones/587f1b42c95fa1e9343f558b7d55252e/dns_records/c0eeac3d89e4abc44c988a70c8656bd5 "HTTP/1.1 200 OK"
2024-08-05 15:24:44,648 - INFO - ✅ DNS record updated successfully! ✅
2024-08-05 15:24:44,648 - INFO - getting list of zones for zone_name: yogajhana.com
2024-08-05 15:24:44,838 - INFO - HTTP Request: GET https://api.cloudflare.com/client/v4